In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install efficientnet

In [ ]:
# !pip install efficientnet

# import library
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from efficientnet.tfkeras import EfficientNetB1
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import Model

In [ ]:
# variabel parameter
dimension = (150, 150)
channel = (3, )
input_shape = dimension + channel
batch_size = 16
epoch = 10

In [ ]:
# parameter data generator
datagenerator = ImageDataGenerator(rescale = 1. / 255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

train_generator = datagenerator.flow_from_directory('/content/drive/My Drive/Colab Notebooks/datasetPitaya/train',
                                                    target_size = dimension,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

test_generator = datagenerator.flow_from_directory('/content/drive/My Drive/Colab Notebooks/datasetPitaya/test',
                                                    target_size = dimension,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

val_generator = datagenerator.flow_from_directory('/content/drive/My Drive/Colab Notebooks/datasetPitaya/validation',
                                                    target_size = dimension,
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

num_class = test_generator.num_classes
labels = train_generator.class_indices.keys()

print(labels)
print(num_class)

Found 1228 images belonging to 6 classes.
Found 311 images belonging to 6 classes.
Found 310 images belonging to 6 classes.
dict_keys(['Antraks', 'Bercak Merah', 'Busuk Batang', 'Busuk Hitam', 'Kudis', 'Mosaik'])
6


In [ ]:
# function data generator
def tf_data_generator(generator, input_shape):
    num_class = generator.num_classes
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types = (tf.float32, tf.float32),
        output_shapes = ([None,
                         input_shape[0],
                         input_shape[1],
                         input_shape[2]],
                        [None, num_class])
    )
    return tf_generator

train_data = tf_data_generator(train_generator, input_shape)
test_data = tf_data_generator(test_generator, input_shape)
val_data = tf_data_generator(val_generator, input_shape)

In [ ]:
# base model
base_model = EfficientNetB1(
    input_shape = input_shape,
    include_top = False,
    weights = 'noisy-student',
    classes = num_class,
)

27017216/27010080 [==============================] - 3s 0us/step


In [ ]:
# custom layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1024, activation = 'relu')(x)

predictions = layers.Dense(num_class, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 125, 125, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 125, 125, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 125, 125, 32) 0           stem_bn[0][0]                    
_______________________________________________________________________________________

In [ ]:
# compile model
print('Compiling Model...')
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

Compiling Model...


In [ ]:
# train Model
history = model.fit(x = train_data,
                   steps_per_epoch = len(train_generator),
                   epochs = epoch,
                   validation_data = val_data,
                   validation_steps = len(val_generator),
                   shuffle = True,
                   verbose = 1)

Epoch 1/10
77/77 [==============================] - 1680s 22s/step - loss: 0.7828 - accuracy: 0.7476 - val_loss: 2.2540 - val_accuracy: 0.5000
Epoch 2/10
77/77 [==============================] - 1087s 14s/step - loss: 0.4837 - accuracy: 0.8339 - val_loss: 0.8961 - val_accuracy: 0.6935
Epoch 3/10
77/77 [==============================] - 1072s 14s/step - loss: 0.4440 - accuracy: 0.8510 - val_loss: 1.0732 - val_accuracy: 0.7161
Epoch 4/10
77/77 [==============================] - 1064s 14s/step - loss: 0.3422 - accuracy: 0.8933 - val_loss: 0.6151 - val_accuracy: 0.8516
Epoch 5/10
77/77 [==============================] - 1063s 14s/step - loss: 0.3254 - accuracy: 0.9039 - val_loss: 0.7150 - val_accuracy: 0.7968
Epoch 6/10
77/77 [==============================] - 1066s 14s/step - loss: 0.2762 - accuracy: 0.9064 - val_loss: 0.6812 - val_accuracy: 0.8129
Epoch 7/10
77/77 [==============================] - 1077s 14s/step - loss: 0.1957 - accuracy: 0.9251 - val_loss: 0.6791 - val_accuracy: 0.7871

In [ ]:
# Save Model
base_path = "/content/drive/My Drive/Colab Notebooks/model"
project_name = "pitaya"
model_name = "model10.h5"
save_model_path = os.path.join(base_path, project_name, model_name)

if os.path.exists(os.path.join(base_path, project_name)) == False:
    os.makedirs(os.path.join(base_path, project_name))
    
print('Saving Model At {}...'.format(save_model_path))
model.save(save_model_path,include_optimizer=False)

Saving Model At /content/drive/My Drive/Colab Notebooks/model/pitaya/model10.h5...


In [ ]:
# Evaluasi Model
loss, acc = model.evaluate(test_data, steps = len(test_generator), verbose = 0)
print('Accuracy pada data training: {:.4f} \nLoss pada data training: {:.4f}'.format(acc, loss), '\n')

loss, acc = model.evaluate(test_data, steps = len(test_generator), verbose = 0)
print('Accuracy pada data test: {:.4f} \nLoss pada data test: {:.4f}'.format(acc, loss), '\n')